## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [2]:
import pandas as pd

df = pd.read_csv('Resources/client_dataset.csv')

df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,item_id,category,subcategory,unit_price,unit_cost,unit_weight,qty,line_number
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,EUD29711-63-6U,decor,wall art,1096.80,762.71,7.50,105,1
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,XDA18116-89-4A,consumables,pens,24.95,15.09,1.49,21,0
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,ABE59463-05-7E,software,project management,13.52,7.86,1.68,39,6
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,ZMM00836-65-0C,consumables,pens,36.42,24.85,1.23,29,3
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,BZX55559-12-3X,consumables,misc,195.10,108.17,46.43,20,1


In [3]:
# View the column names in the data
df.columns


Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')

In [4]:
# Use the describe function to gather some basic statistics
df.describe()


,client_id,order_id,order_week,order_year,unit_price,unit_cost,unit_weight,qty,line_number
count,54639.000000,5.463900e+04,54639.000000,54639.000000,54639.000000,54639.000000,54639.000000,5.463900e+04,54639.000000
mean,54837.869416,5.470190e+06,11.359139,2022.993064,136.267207,99.446073,5.004116,5.702646e+02,2.979667
std,25487.438231,2.599807e+06,7.023499,0.082997,183.873135,133.164267,5.326599,1.879552e+04,2.436320
min,10033.000000,1.000886e+06,1.000000,2022.000000,0.010000,0.010000,0.000000,0.000000e+00,0.000000
25%,33593.000000,3.196372e+06,6.000000,2023.000000,20.800000,14.840000,1.450000,3.200000e+01,1.000000
50%,53305.000000,5.496966e+06,11.000000,2023.000000,68.310000,49.890000,3.240000,6.800000e+01,3.000000
75%,78498.000000,7.733869e+06,17.000000,2023.000000,173.160000,125.570000,6.890000,1.700000e+02,5.000000
max,99984.000000,9.998480e+06,52.000000,2023.000000,1396.230000,846.270000,46.430000,3.958244e+06,9.000000


In [5]:
# Use this space to do any additional research
# and familiarize yourself with the data.
df.shape


(54639, 18)

In [6]:
categories = df["category"].unique()
print(categories)

['decor' 'consumables' 'software' 'furniture' 'electronics']


In [7]:
# What three item categories had the most entries?
totals = {}
categories = df["category"].unique()
for category in categories:
    totals[category] = df.loc[df['category'] == category, 'category'].count()

categories_data = pd.Series(totals, name='category').sort_values(ascending=False) 
categories_data.head(3)
#print(totals)



consumables    23538
furniture      11915
software        8400
Name: category, dtype: int64

In [8]:
df['subcategory'].unique()

array(['wall art', 'pens', 'project management', 'misc', 'plants',
       'kitchen supplies', 'paper products', 'seating', 'computers',
       'stands', 'tables', 'bathroom supplies', 'audio visual',
       'accounting', 'logistics', 'telecom', 'desks', 'cabling'],
      dtype=object)

In [9]:
max_consumable_subs = df.loc[df['category'] == 'consumables',['category','subcategory']].groupby(['subcategory']).count().max()
max_consumable_subs

category    6424
dtype: int64

In [10]:
# For the category with the most entries,
# which subcategory had the most entries?
df.loc[df['category'] == 'consumables', ['category','subcategory']].groupby('subcategory').count().max()


category    6424
dtype: int64

In [11]:
# Which five clients had the most entries in the data?
totals = {}
client_ids = df["client_id"].unique()
for client_id in client_ids:
    totals[client_id] = df.loc[df['client_id'] == client_id, 'client_id'].count()

categories_data = pd.Series(totals, name='client_id').sort_values(ascending=False) 
categories_data.head(5)


33615    220
66037    211
46820    209
38378    207
24741    207
Name: client_id, dtype: int64

In [12]:
# Store the client ids of those top 5 clients in a list.
top5_client_ids = categories_data.head().index.to_list()
top5_client_ids


[33615, 66037, 46820, 38378, 24741]

In [13]:
# How many total units (the qty column) did the
# client with the most entries order order?
total_qty = df.loc[df['client_id'] == 33615, 'qty'].sum() 
total_qty


64313

## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [14]:
def get_unit_price(row):
    return float(row['unit_price'] ) * int(row['qty'])
    
df['line_subtotal'] = df.apply(get_unit_price, axis=1)
df[['unit_price','qty','line_subtotal']]

,unit_price,qty,line_subtotal
0,1096.80,105,115164.00
1,24.95,21,523.95
2,13.52,39,527.28
3,36.42,29,1056.18
4,195.10,20,3902.00
...,...,...,...
54634,83.13,33,2743.29
54635,206.59,47,9709.73
54636,65.66,475,31188.50
54637,1.48,112,165.76


In [15]:
# Create a column that calculates the 
# subtotal for each line using the unit_price
# and the qty
def get_unit_price(row):
    return float(row['unit_price'] ) * int(row['qty'])
    
df['line_subtotal'] = df.apply(get_unit_price, axis=1)
df[['unit_price','qty','line_subtotal']]


,unit_price,qty,line_subtotal
0,1096.80,105,115164.00
1,24.95,21,523.95
2,13.52,39,527.28
3,36.42,29,1056.18
4,195.10,20,3902.00
...,...,...,...
54634,83.13,33,2743.29
54635,206.59,47,9709.73
54636,65.66,475,31188.50
54637,1.48,112,165.76


In [16]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound
# for orders over 50 pounds and $10 per
# pound for items 50 pounds or under.
def get_shipping_price(row):
    qty = int(row['qty'])
    total_weight = float(row['unit_weight']) * qty

    if total_weight > 50:        
        shipping_price = total_weight * 7
    else:
        shipping_price = total_weight * 10

    return shipping_price

def get_total_weight(row):
    return float(row['unit_weight']) * int(row['qty'])

df['shipping_price'] = df.apply(get_shipping_price, axis=1)
df['total_weight'] = df.apply(get_total_weight, axis=1)

df[['unit_price','unit_weight','qty','total_weight','shipping_price']]


,unit_price,unit_weight,qty,total_weight,shipping_price
0,1096.80,7.50,105,787.50,5512.50
1,24.95,1.49,21,31.29,312.90
2,13.52,1.68,39,65.52,458.64
3,36.42,1.23,29,35.67,356.70
4,195.10,46.43,20,928.60,6500.20
...,...,...,...,...,...
54634,83.13,2.25,33,74.25,519.75
54635,206.59,11.70,47,549.90,3849.30
54636,65.66,4.16,475,1976.00,13832.00
54637,1.48,18.04,112,2020.48,14143.36


In [73]:
# Create a column for the total price
# using the subtotal and the shipping price
# along with a sales tax of 9.25%
def get_total_price(x):
    #unit_price = x['unit_price'] 
    #qty = x['qty']
    line_subtotal = x['line_subtotal']
    shipping_price = x['shipping_price']
    sales_tax = 0.0925

    #subtotal = (unit_price * qty) + shipping_price
    subtotal = line_subtotal + shipping_price
    return format((subtotal * sales_tax) + subtotal, '.2f')
    
df["line_price"] = df.apply(get_total_price, axis=1).astype('float')
df[['line_subtotal','shipping_price','line_price']]


,line_subtotal,shipping_price,line_price
0,115164.00,5512.50,131839.08
1,523.95,312.90,914.26
2,527.28,458.64,1077.12
3,1056.18,356.70,1543.57
4,3902.00,6500.20,11364.40
...,...,...,...
54634,2743.29,519.75,3564.87
54635,9709.73,3849.30,14813.24
54636,31188.50,13832.00,49184.90
54637,165.76,14143.36,15632.71


In [71]:
# Create a column for the cost
# of each line using unit cost, qty, and
# shipping price (assume the shipping cost
# is exactly what is charged to the client).
def get_total_cost(x):
    unit_cost = x['unit_cost'] 
    qty = x['qty']
    shipping_cost = x['shipping_price']   

    subtotal = (unit_cost * qty) + shipping_cost
    return format(subtotal, '.2f')

df["line_cost"] = df.apply(get_total_cost, axis=1).astype('float')
df.head()


,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_cost,unit_weight,qty,line_number,line_subtotal,shipping_price,total_weight,line_price,line_cost,line_profit
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,762.71,7.50,105,1,115164.00,5512.50,787.50,131839.08,85597.05,46242.03
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,15.09,1.49,21,0,523.95,312.90,31.29,914.26,629.79,284.47
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,7.86,1.68,39,6,527.28,458.64,65.52,1077.12,765.18,311.94
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,...,24.85,1.23,29,3,1056.18,356.70,35.67,1543.57,1077.35,466.22
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,...,108.17,46.43,20,1,3902.00,6500.20,928.60,11364.40,8663.60,2700.80


In [77]:
# Create a column for the profit of
# each line using line cost and line price
df['line_profit'] = df.apply(lambda x: (x['line_price'] - x['line_cost']), axis=1).astype('float')
df.head()


,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_cost,unit_weight,qty,line_number,line_subtotal,shipping_price,total_weight,line_price,line_cost,line_profit
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,762.71,7.50,105,1,115164.00,5512.50,787.50,131839.08,85597.05,46242.03
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,15.09,1.49,21,0,523.95,312.90,31.29,914.26,629.79,284.47
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,7.86,1.68,39,6,527.28,458.64,65.52,1077.12,765.18,311.94
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,...,24.85,1.23,29,3,1056.18,356.70,35.67,1543.57,1077.35,466.22
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,...,108.17,46.43,20,1,3902.00,6500.20,928.60,11364.40,8663.60,2700.80


## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [79]:
# Check your work using the totals above
order_id = 2742071
total = df.loc[df['order_id'] == order_id][['line_price']].sum()
print(f"Order ID {order_id} had a total price of ${total.iloc[0]:,.2f}")

order_id = 2173913
total = df.loc[df['order_id'] == order_id][['line_price']].sum()
print(f"Order ID {order_id} had a total price of ${total.iloc[0]:,.2f}")

order_id = 6128929
total = df.loc[df['order_id'] == order_id][['line_price']].sum()
print(f"Order ID {order_id} had a total price of ${total.iloc[0]:,.2f}")

Order ID 2742071 had a total price of $152,811.89
Order ID 2173913 had a total price of $162,388.71
Order ID 6128929 had a total price of $923,441.25


## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [150]:
totals_by_top5 = []
for client_id in top5_client_ids:
    totals = df.loc[df['client_id'] == client_id]['line_price'].sum()    
    totals_by_top5.append({
        'client_id': client_id,
        'totals': format(totals, ',.2f')})       
    
totals_by_top5

[{'client_id': 33615, 'totals': '8,377,308.52'},
 {'client_id': 66037, 'totals': '10,259,514.80'},
 {'client_id': 46820, 'totals': '9,743,794.33'},
 {'client_id': 38378, 'totals': '12,906,550.87'},
 {'client_id': 24741, 'totals': '82,268,892.04'}]

In [16]:
# How much did each of the top 5 clients by quantity
# spend? Check your work from Part 1 for client ids.
totals_by_top5 = []
for client_id in top5_client_ids:
    totals = df.loc[df['client_id'] == client_id]['line_price'].sum()
    totals_by_top5.append('{' +
        str(client_id) + ': ' +
        format(totals, ',.2f') + '}')
    
print(totals_by_top5)


33615: $8377308.52
66037: $10259514.79
46820: $9743794.36
38378: $12906550.87
24741: $82268892.02


In [141]:
summary_df['client_id'] = top5_client_ids
summary_df['qty']

,0
0,"{33615: 8,377,308.52}"
1,"{66037: 10,259,514.80}"
2,"{46820: 9,743,794.33}"
3,"{38378: 12,906,550.87}"
4,"{24741: 82,268,892.04}"


In [144]:
summary_df = df.loc[df['client_id'] in top5_client_ids][['client_id','qty','shipping_price','line_price','line_cost','line_profit']]
summary_df

AttributeError: 'list' object has no attribute 'item'

In [145]:
top5_client_ids

[33615, 66037, 46820, 38378, 24741]

In [17]:
# Create a summary DataFrame showing the totals for the
# for the top 5 clients with the following information:
# total units purchased, total shipping price,
# total revenue, and total profit. Sort by total profit.
summary_df = pd.DataFrame(columns=['client_id','qty','shipping_price','line_price','line_cost','line_profit'])
for i in range(5):
    summary_df.loc[i] = 

df.loc[df['client_id'] in top5_client_ids][['client_id','qty','shipping_price','line_price','line_cost','line_profit']]
summary_df


,client_id,qty,shipping_price,line_price,line_cost,line_profit
4,24741,239862,5126448.37,82268892.02,45688899.71,36579992.31
3,38378,73667,3429455.40,12906550.87,9634720.98,3271829.89
1,66037,43018,1395151.85,10259514.79,7004482.98,3255031.81
2,46820,75768,1601448.84,9743794.36,7007191.64,2736602.72
0,33615,64313,1828984.89,8377308.52,6175313.91,2201994.61


In [18]:
# Format the data and rename the columns
# to names suitable for presentation.
# Currency should be in millions of dollars.



In [19]:
# Sort the updated data by "Total Profit" form highest to lowest


,Client ID,Units,Shipping,Total Revenue,Total Cost,Total Profit
4,24741,239862,$5.13M,$82.27M,$45.69M,$36.58M
3,38378,73667,$3.43M,$12.91M,$9.63M,$3.27M
1,66037,43018,$1.40M,$10.26M,$7.00M,$3.26M
2,46820,75768,$1.60M,$9.74M,$7.01M,$2.74M
0,33615,64313,$1.83M,$8.38M,$6.18M,$2.20M
